# YOLO labeling format

Most annotation platforms support export at YOLO labeling format, providing one annotations text file per image. Each text file contains one bounding-box (BBox) annotation for each of the objects in the image. The annotations are normalized to the image size, and lie within the range of 0 to 1. They are represented in the following format:

$< object-class-ID> <X center> <Y center> <Box width> <Box height>$

![annotation box](https://blog.paperspace.com/content/images/2021/03/image-25.png)

In [1]:
import pandas as pd
import numpy as np
import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

In [3]:
annotation = pd.read_excel("Preprocessing/Annotation_Boxes.xlsx")

In [4]:
annotation.head()

,Patient ID,Start Row,End Row,Start Column,End Column,Start Slice,End Slice
0,Breast_MRI_001,234,271,308,341,89,112
1,Breast_MRI_002,251,294,108,136,59,72
2,Breast_MRI_003,351,412,82,139,96,108
3,Breast_MRI_004,262,280,193,204,86,95
4,Breast_MRI_005,188,213,138,178,76,122


## Normalize Coordinates to 0-1

In [5]:
def load_normalize(image_path, normalized=True):
    image = Image.open(image_path)
    image= np.array(image).astype(np.float32)
    if len(image.shape) == 3:
        image = image.sum(axis=2)
    if normalized:
        return image / 255.0
    else:
        return image

In [6]:
x_data = []
scale = {}


all_images = glob.glob('/workspaces/breast-tumor-detection/Data/pos/*.png')

for image_path in all_images:
    id = image_path.split('/')[-1].split('.')[0].split('-')[-1]
    img = load_normalize(image_path)
    x_data.append(img)
    scale[id] = img.shape

In [7]:
size = pd.DataFrame(scale.items(), columns=['id', 'size'])
size["id"] = size["id"].apply(lambda x: x[:6]+"_"+x[6:9]+"_"+x[9:12])
annotation = pd.merge(annotation, size.set_index('id'), left_on='Patient ID', right_index=True)
annotation.head()

,Patient ID,Start Row,End Row,Start Column,End Column,Start Slice,End Slice,size
0,Breast_MRI_001,234,271,308,341,89,112,"(448, 448)"
1,Breast_MRI_002,251,294,108,136,59,72,"(512, 512)"
2,Breast_MRI_003,351,412,82,139,96,108,"(512, 512)"
3,Breast_MRI_004,262,280,193,204,86,95,"(512, 512)"
4,Breast_MRI_005,188,213,138,178,76,122,"(448, 448)"


In [8]:
assert len(annotation) == len(x_data)

In [9]:
annotation['start_row_scaled'] = np.zeros(len(annotation))
annotation['start_column_scaled'] = np.zeros(len(annotation))
annotation['end_row_scaled'] = np.zeros(len(annotation))
annotation['end_column_scaled'] = np.zeros(len(annotation))

In [10]:
def add_scaled_boxes(row,target_size=(512,512)):
    y_ = row['size'][0]
    x_ = row['size'][1]
    y_scale = target_size[0]/y_
    x_scale = target_size[1]/x_
    # df['start_row_scaled'] = df['Start Row'].apply(lambda x: round(x*y_scale/target_size[0],2))
    # df['end_row_scaled'] = df['End Row'].apply(lambda x: round(x*y_scale/target_size[0],2))
    # df['start_column_scaled'] = df['Start Column'].apply(lambda x: round(x*x_scale/target_size[1],2))
    # df['end_column_scaled'] = df['End Column'].apply(lambda x: round(x*x_scale/target_size[1],2))
    # row['start_row_scaled'] = row['Start Row'].apply(lambda x: round(x*y_scale,2))
    # row['end_row_scaled'] = row['End Row'].apply(lambda x: round(x*y_scale,2))
    # row['start_column_scaled'] = row['Start Column'].apply(lambda x: round(x*x_scale,2))
    # row['end_column_scaled'] = row['End Column'].apply(lambda x: round(x*x_scale,2))
    row['start_row_scaled'] = round(row['Start Row']*y_scale/target_size[0],2)
    row['end_row_scaled'] = round(row['End Row']*y_scale/target_size[0],2)
    row['start_column_scaled'] = round(row['Start Column']*x_scale/target_size[1],2)
    row['end_column_scaled'] = round(row['End Column']*x_scale/target_size[1],2)
    return row


In [11]:
annotation = annotation.apply(add_scaled_boxes, axis=1)

In [12]:
annotation['x_center'] = annotation.apply(lambda x: round((x['start_column_scaled'] + x['end_column_scaled'])/2,2), axis=1)
annotation['y_center'] = annotation.apply(lambda x: round((x['start_row_scaled'] + x['end_row_scaled'])/2,2), axis=1)
annotation['width'] = annotation.apply(lambda x: round(x['end_column_scaled'] - x['start_column_scaled'],2), axis=1)
annotation['height'] = annotation.apply(lambda x: round(x['end_row_scaled'] - x['start_row_scaled'],2), axis=1)

In [13]:
annotation.loc[173]

Patient ID             Breast_MRI_174
Start Row                         345
End Row                           369
Start Column                      444
End Column                        461
Start Slice                        52
End Slice                          60
size                       (512, 512)
start_row_scaled                 0.67
start_column_scaled              0.87
end_row_scaled                   0.72
end_column_scaled                 0.9
x_center                         0.89
y_center                          0.7
width                            0.03
height                           0.05
Name: 173, dtype: object

In [14]:
annotation_txt = annotation[['Patient ID', 'x_center', 'y_center', 'width', 'height']]
annotation_txt["Patient ID"] = np.ones(len(annotation_txt))
annotation_txt.head()

/tmp/ipykernel_9919/2084676687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation_txt["Patient ID"] = np.ones(len(annotation_txt))


,Patient ID,x_center,y_center,width,height
0,1.0,0.72,0.56,0.07,0.08
1,1.0,0.24,0.53,0.06,0.08
2,1.0,0.22,0.74,0.11,0.11
3,1.0,0.39,0.53,0.02,0.04
4,1.0,0.35,0.45,0.09,0.06


In [16]:
annotation.to_csv('annotations.csv', index=False)

In [15]:
annotation_txt.to_csv('annotations.txt', sep='\t', index=False, header=None)